In [3]:
import numpy as np
import pandas as pd

from pretrained_embedding import get_input_plus_embedding_vectors
from grid_search_three_subsets import grid_search

from keras.models import Model

from keras.layers.embeddings import Embedding
from keras.layers import Input, Dense, Activation, Flatten, Conv1D, MaxPooling1D
from tensorflow.keras import regularizers

In [2]:
def create_model(vocabulary_length, max_length_sequence, emb_dim, embedding_vectors,
                 filters, kernel_size, dense_units, l2_kernel):
    
    X_input = Input(shape = (max_length_sequence, ))
    embedding_layer = Embedding(input_dim = vocabulary_length, output_dim = emb_dim, weights=[embedding_vectors],
                                trainable = False)(X_input)
    
    X = Conv1D(filters = filters, kernel_size = kernel_size, activation = 'relu',
              kernel_regularizer = regularizers.l2(l2_kernel))(embedding_layer)
    X = MaxPooling1D(pool_size = 2)(X)
    X = Flatten()(X)
    X = Dense(units = dense_units, activation = 'relu')(X)
    X = Dense(units = 1, activation = 'sigmoid')(X)
                          
    model = Model(inputs = X_input, outputs = X)
                          
    return(model)

In [4]:
english_dataset = pd.read_csv('../data/Merged/english_dataset.csv')
english_dataset

,text,label
0,Donald Trump just couldn t wish all Americans ...,1
1,House Intelligence Committee Chairman Devin Nu...,1
2,"On Friday, it was revealed that former Milwauk...",1
3,"On Christmas day, Donald Trump announced that ...",1
4,Pope Francis used his annual Christmas Day mes...,1
...,...,...
51228,The State Department told the Republican Natio...,0
51229,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,1
51230,Anti-Trump Protesters Are Tools of the Oligar...,1
51231,"ADDIS ABABA, Ethiopia —President Obama convene...",0


In [5]:
vocabulary_length = 10000
max_length_sequence = 1500
emb_dim = 300
language = 'english'
embedding_file_path = '../data/GloVe_Embedding/glove.6B.300d.txt'

In [6]:
X, df, embedding_vectors = get_input_plus_embedding_vectors(english_dataset, embedding_file_path, 
                                                           vocabulary_length, max_length_sequence, emb_dim, language)

In [7]:
#model = create_model(vocabulary_length, max_length_sequence, emb_dim, embedding_vectors, filters = 16, kernel_size = 10, dense_units = 12, l2_kernel = 0)
#model.summary()

In [8]:
models = []

model_0 = create_model(vocabulary_length, max_length_sequence, emb_dim, embedding_vectors, filters = 16, kernel_size = 10, dense_units = 4, l2_kernel = 0)
model_1 = create_model(vocabulary_length, max_length_sequence, emb_dim, embedding_vectors, filters = 16, kernel_size = 10, dense_units = 8, l2_kernel = 0)
model_2 = create_model(vocabulary_length, max_length_sequence, emb_dim, embedding_vectors, filters = 16, kernel_size = 10, dense_units = 12, l2_kernel = 0)
model_3 = create_model(vocabulary_length, max_length_sequence, emb_dim, embedding_vectors, filters = 16, kernel_size = 10, dense_units = 12, l2_kernel = 0.01)


models.append(model_0)
models.append(model_1)
models.append(model_2)
models.append(model_3)

In [9]:
Y = df.label.values
epochs = 7
batch_size = 32
iterations = 5
test_size = 0.2

In [ ]:
best_model_index, test_acc = grid_search(X, Y, models, epochs, batch_size, iterations, test_size)
print('best_model_index: ' + str(best_model_index), 'test acc: ' + str(test_acc))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/7
32788/32788 [==============================] - 449s 14ms/step - loss: 0.1332 - accuracy: 0.9499
Epoch 2/7
32788/32788 [==============================] - 440s 13ms/step - loss: 0.0555 - accuracy: 0.9805
Epoch 3/7
32788/32788 [==============================] - 436s 13ms/step - loss: 0.0260 - accuracy: 0.9914
Epoch 4/7
32788/32788 [==============================] - 421s 13ms/step - loss: 0.0118 - accuracy: 0.9968
Epoch 5/7
32788/32788 [==============================] - 421s 13ms/step - loss: 0.0073 - accuracy: 0.9978
Epoch 6/7
32788/32788 [==============================] - 421s 13ms/step - loss: 0.0075 - accuracy: 0.9973
Epoch 7/7
8198/8198 [==============================] - 35s 4ms/step
Epoch 1/7
32788/32788 [==============================] - 418s 13ms/step - loss: 0.1378 - accuracy: 0.9450
Epoch 2/7
32788/32788 [==============================] - 422s 13ms/step - loss: 0.0511 - accura

#### Train Model with English Dataset and Evaluate with Translated Dataset